In [29]:
def read_file(file_name):
	with open(file_name, "r") as f:
		data = f.read().splitlines()
	return data

In [30]:
EXAMPLE_FILE = "example"
INPUT_FILE = "input"

In [31]:
def cab_distance(start, destination):
	return abs(start[0] - destination[0]) + abs(start[1] - destination[1])

In [32]:
def prepare_data(file_name):
	sensor_map = {}
	beacons = []
	data = read_file(file_name)
	for line in data:
		line = line.split(' ')
		sensor_x = int(line[2][2:-1])
		sensor_y = int(line[3][2:-1])
		beacon_x = int(line[8][2:-1])
		beacon_y = int(line[9][2:])
		distance = cab_distance((sensor_x, sensor_y), (beacon_x, beacon_y))
		sensor_map[(sensor_x, sensor_y)] = distance	
		beacons.append((beacon_x, beacon_y))
	return (sensor_map, beacons)

sensor_map, beacons = prepare_data(EXAMPLE_FILE)
sensor_map, beacons

({(2, 18): 7,
  (9, 16): 1,
  (13, 2): 3,
  (12, 14): 4,
  (10, 20): 4,
  (14, 17): 5,
  (8, 7): 9,
  (2, 0): 10,
  (0, 11): 3,
  (20, 14): 8,
  (17, 20): 6,
  (16, 7): 5,
  (14, 3): 1,
  (20, 1): 7},
 [(-2, 15),
  (10, 16),
  (15, 3),
  (10, 16),
  (10, 16),
  (10, 16),
  (2, 10),
  (2, 10),
  (2, 10),
  (25, 17),
  (21, 22),
  (15, 3),
  (15, 3),
  (15, 3)])

In [33]:
def find_extremas(sensor_map):
	max_distance = max(sensor_map.values())
	x_coords = [x for x,y in sensor_map.keys()]
	return (min(x_coords) - max_distance, max(x_coords) + max_distance)

find_extremas(sensor_map)

(-10, 30)

In [34]:
def reachable_from_sensor(pos, sensor_map):
	for sensor_pos, distance in sensor_map.items():
		if cab_distance(sensor_pos, pos) <= distance:
			return True
	return False

reachable_from_sensor((-3,10), sensor_map)
reachable_from_sensor((2,26), sensor_map)

False

In [35]:
def solve_part1(file_name, row):
	count = 0
	sensor_map, beacons = prepare_data(file_name)
	min_x, max_x = find_extremas(sensor_map)
	for x in range(min_x, max_x + 1):
		if (x, row) in beacons:
			continue
		if reachable_from_sensor((x, row), sensor_map):
			count += 1
	return count

print(f"Example: {solve_part1(EXAMPLE_FILE, 10)}")
print(f"Puzzle: {solve_part1(INPUT_FILE, 2_000_000)}")

Example: 26
Puzzle: 6078701


In [36]:
def tuning_frequency(x, y):
	return x * 4_000_000 + y

In [37]:
def neumann_neighbourhood_border(pos, distance):
	result = []
	x, y = pos
	for step in range(distance + 1):
		result.append((x + step, y - distance - 1 + step))
		result.append((x - step, y + distance + 1 - step))
		result.append((x - distance - 1 + step, y - step))
		result.append((x + distance + 1 - step, y + step))
	return result

neumann_neighbourhood_border((0,0), 2)

[(0, -3),
 (0, 3),
 (-3, 0),
 (3, 0),
 (1, -2),
 (-1, 2),
 (-2, -1),
 (2, 1),
 (2, -1),
 (-2, 1),
 (-1, -2),
 (1, 2)]

In [39]:


def solve_part2(file_name, max_coord):
	sensor_map, beacons = prepare_data(file_name)
	checked = set()
	for sensor_pos, distance in sensor_map.items():
		for pos in neumann_neighbourhood_border(sensor_pos, distance):
			x, y = pos
			if x not in range(max_coord + 1) or y not in range(max_coord + 1) or pos in checked or pos in beacons:
				continue
			# TODO: remove sensor that create the hood
			if not reachable_from_sensor(pos, sensor_map):
				print(pos, reachable_from_sensor(pos, sensor_map))
				return tuning_frequency(*pos)
		checked.add(pos)
	return (-1, -1)

print(f"Example: {solve_part2(EXAMPLE_FILE, 20)}")
print(f"Puzzle: {solve_part2(INPUT_FILE, 4_000_000)}")

(14, 11) False
Example: 56000011
(3141837, 3400528) False
Puzzle: 12567351400528
